# **How Accurate are GPT Metrics Compared to Human Analysis?**

### Overview
In this demonstration, you will compare GPT4s assessment of Coherence against human graded samples. This demo uses the SummEval dataset, SummEval contains LLM generated summarizations that have been human graded on Coherence. Coherence measures the quality of all sentences in a model's predicted answer and how they fit together naturally.

You will utilize Azure PromptFlow to generate Coherence evaluations using GPT4. Using PromptFlow you will use two Coherence prompts - a standard template and a template using 'emotion' as referenced in the linked paper below. You will then use this notebooks to compare and analyze the GPT4, GPT4 with emotion, and human graded coherence scores.

 **_Go Deeper_**  
- [Are Large Language Model-based Evaluators the Solution to Scaling Up Multilingual Evaluation?](https://ar5iv.labs.arxiv.org/html/2309.07462)
- [GptEval: NLG Evaluation using Gpt-4 with Better Human Alignment](https://ar5iv.labs.arxiv.org/html/2303.16634)
- [EmotionPrompt: Leveraging Psychology for Large Language Models Enhancement via Emotional Stimulus](https://arxiv.org/pdf/2307.11760v3.pdf)
- SummEval: Re-evaluating Summarization Evaluation [[Paper]](https://arxiv.org/pdf/2007.12626.pdf) / [[Repository]](https://github.com/Yale-LILY/SummEval#data)
### Prerequisites
Ensure that your environment is setup by completing the steps outlines in [0_setup.ipynb](./0_setup.ipynb)  
_Optional_: For an overview of GPT based metrics, please see [1_gpt_evaluation.ipynb](./1_gpt_evaluation.ipynb)

### Step 1: Examine Input Data
The SummEval dataset provides human labeled analysis on LLM generated summaries.The annotations include summaries generated by 16 models from 100 source news articles (~156000 examples in total). You will use a subset of ~1000 samples.
Each of the summaries was annotated by 5 indepedent crowdsource workers and 3 independent experts (8 annotations in total).
Summaries were evaluated across 4 dimensions: coherence, consistency, fluency, relevance.
Each source news article comes with the original reference from the CNN/DailyMail dataset and 10 additional crowdsources reference summaries.  
  
Take a look at the data [summEval_human_labeled.jsonl](../data/inputs/summEval_human_labeled.jsonl)  

_Note: Human labels are subjective and are **not** perfect, comparison is for benchmarking purposes and should be considered separate from result quality_

### Step 2: Run Evaluation Pipeline

**IMPORTANT:** Be sure to analyze and expirament with the [gpt_eval_benchmark](../src/promptflow/evaluation_flows/gpt_eval_benchmark/) PromptFlow used in this step.  
  
  _This step may take upt to 10 minutes to complete_

In [2]:
from promptflow import PFClient

# PFClient can help manage your runs and connections.
pf = PFClient()

# Define Flows and Data
eval_flow = "../src/promptflow/evaluation_flows/gpt_eval_benchmark" # set flow directory
data = "../data/inputs/summEval_human_labeled.jsonl" # set the data file

# Run evaluation flow to evaluate chat results
eval_run = pf.run(
    flow=eval_flow,
    data=data,
    stream=False,
    column_mapping={  # map the url field from the data to the url input of the flow
      "expert_annotaions": "${data.expert_annotations}",
      "turker_annotations": "${data.turker_annotations}",
      "response": "${data.decoded}",
    }
)


: 